<a href="https://colab.research.google.com/github/takumi0706/IntroductionMLByPython/blob/main/Chapter3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
pip install chainer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for chainer: filename=chainer-7.8.1-py3-none-any.whl size=967716 sha256=9872c461f265d1dce3fd44aa62abceb7b5153055bb74cd50063e44ac06d85d01
  Stored in directory: /root/.cache/pip/wheels/c4/95/6a/16014db6f761c4e742755b64aac60dbe142da1df6c5919f790
Successfully built chainer


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn.datasets as ds

In [17]:
Iris = ds.load_iris()
xdata = Iris.data.astype(np.float32)
tdata = Iris.target.astype(np.int32)

In [18]:
D,N = xdata.shape

In [19]:
Dtrain = D//2
# permutation == ランダムにD=150の数字をランダムに並び替える
index = np.random.permutation(range(D))
xtrain = xdata[index[0:Dtrain],:]
ttrain = tdata[index[0:Dtrain]]
xtest = xdata[index[Dtrain:D],:]
ttest = tdata[index[Dtrain:D]]

In [21]:
import chainer.optimizers as Opt
import chainer.functions as F
import chainer.links as L
from chainer import Variable,Chain,config

In [23]:
# 最大値抽出
C = np.max(tdata)+1
NN = Chain(l1 = L.Linear(N,3),l2 = L.Linear(3,3), l3 = L.Linear(3,C))

In [24]:
def model(x):
  h = NN.l1(x)
  h = F.sigmoid(h)
  h = NN.l2(x)
  h = F.sigmoid(h)
  y = NN.l3(x)
  return y